# Step 1: Upload the Images ZIP file
This step prompts you to upload a ZIP file containing images from your computer to Colab. Once the file is uploaded, its name is stored in the `zip_file_name` variable for further processing.

In [ ]:
from google.colab import files
uploaded = files.upload()

zip_file_name = next(iter(uploaded.keys()))

# Step 2: Import necessary libraries and then Define the path to the uploaded zip file
In this step,import the necessary libraries to handle files and data, then define the path to the uploaded ZIP file (which contains images). A new folder called "images" is created, and the contents of the ZIP file are extracted into this folder for further processing.

In [ ]:
import os
import zipfile
import pandas as pd
from shutil import copyfile, make_archive


zip_file_path = zip_file_name
extracted_folder_path = 'images/'

os.makedirs(extracted_folder_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Step 3:Define the metadata for each buoy type
In this step, hve define metadata for eight different types of navigation buoys. For each buoy type, the metadata includes a label describing what the buoy represents and a detailed description, explaining its purpose and safety instructions. This information will be used to classify and explain the buoy types in later steps.

In [ ]:

buoy_metadata = {
    'port': {
        'label': 'Region A red port lateral mark',
        'description': 'Port is left and associated with the colour red and cylinder shape in Region A. On the way into a harbour we will keep the red marks to the left (port) side of our ship or boat to travel safely within the channel opposite when exiting the harbour or channel.'
    },
    'stbd': {
        'label': 'Region A Green starboard lateral mark',
        'description': 'Starboard is right and associated with the colour green and cone shape in Region A. On the way into a harbour we will keep the green marks to the right (starboard) side of our ship or boat to travel safely within the channel opposite when exiting the harbour or channel.'
    },
    'south': {
        'label': 'South Cardinal Buoy',
        'description': 'A South Cardinal indicates that the safest water is to the South. You should pass the South side of the mark.'
    },
    'north': {
        'label': 'North Cardinal Buoy',
        'description': 'A North Cardinal indicates that the safest water is to the North. You should pass the North side of the mark.'
    },
    'east': {
        'label': 'East Cardinal Buoy',
        'description': 'An East Cardinal indicates that the safest water is to the East. You should pass the East side of the mark.'
    },
    'west': {
        'label': 'West Cardinal Buoy',
        'description': 'A West Cardinal indicates that the safest water is to the West. You should pass the West side of the mark.'
    },
    'isolateddanger': {
        'label': 'Isolated Danger Mark',
        'description': 'Isolated danger marks show where there is an isolated danger that has navigable water all around it (for example, an isolated shoal, rock, or wreck) – but don\'t pass too close. Keep a safe distance.'
    },
    'safewater': {
        'label': 'Fairway Buoy / Safe Water Mark Buoy',
        'description': 'Safe Water marks serve to indicate that there is navigable water all around the mark and do not mark a danger. Safe water marking can be used for fairway, mid-channel, centre line, or landfall marks. This mark may also be used to indicate channel entrance, port or estuary approach, or landfall.'
    }
}


# Step 4:List the extracted files
This step lists all the files extracted from the ZIP file in the "images" folder. The file names are stored in the `image_files` list, which prepares the images for further processing. The `data` list is initialized for later use, likely for storing relevant information about the images.

In [ ]:

image_files = os.listdir(os.path.join(extracted_folder_path, 'images'))

data = []

In [ ]:
# Step 5:Populate the data list with the image metadata
for image_file in image_files:
    for key, value in buoy_metadata.items():
        if key in image_file.replace('.', '').lower():
            data.append({
                'Filename': image_file,
                'Label': value['label'],
                'Description': value['description']
            })

df = pd.DataFrame(data)

#Step 5: Verify if all buoy types are included
This step verifies whether all buoy types defined in the metadata are present in the dataset. It checks the unique labels in the dataset and compares them to the buoy labels defined. If all buoy types are included, it confirms otherwise, it alerts you that some buoy types are missing.

In [ ]:

included_buoys = df['Label'].unique()
all_buoys_included = all(buoy['label'] in included_buoys for buoy in buoy_metadata.values())
print("Included buoy types in the dataset:")
print(included_buoys)

if all_buoys_included:
    print("All buoy types are included in the dataset.")
else:
    print("Not all buoy types are included in the dataset.")

#Step 6: Save the DataFrame to a CSV file
This step saves the DataFrame `df` as a CSV file, creates a new folder called "buoy_dataset," and copies the image files and the CSV file into that folder. Then, the entire "buoy_dataset" folder is compressed into a ZIP file, and the path to this newly created ZIP file is printed for reference.

In [ ]:
#Step 6: Save the DataFrame to a CSV file
csv_file_path = os.path.join(extracted_folder_path, 'buoy_dataset_with_isolated_danger.csv')
df.to_csv(csv_file_path, index=False)

dataset_folder_path = 'buoy_dataset'
os.makedirs(dataset_folder_path, exist_ok=True)

for image_file in image_files:
    copyfile(os.path.join(extracted_folder_path, 'images', image_file), os.path.join(dataset_folder_path, image_file))
copyfile(csv_file_path, os.path.join(dataset_folder_path, 'buoy_dataset_with_isolated_danger.csv'))

dataset_zip_path = make_archive(dataset_folder_path, 'zip', dataset_folder_path)

print(f'Dataset ZIP file created at: {dataset_zip_path}')


Saving images.zip to images.zip
Included buoy types in the dataset:
['Region A red port lateral mark' 'South Cardinal Buoy'
 'West Cardinal Buoy' 'Fairway Buoy / Safe Water Mark Buoy'
 'Region A Green starboard lateral mark' 'Isolated Danger Mark'
 'North Cardinal Buoy' 'East Cardinal Buoy']
All buoy types are included in the dataset.
Dataset ZIP file created at: /content/buoy_dataset.zip
